# Spam Classifier 
**Algorithm Used:** Naive Bayes (Multinomial)<br>
**Date:** 28/09/2019<br>
**Author:** Pranay Saha

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
message_data= pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',encoding='latin-1')
message_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
message_data= message_data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)

In [4]:
message_data= message_data.rename(columns = {'v1':'Spam/Not_Spam','v2':'message'})

In [5]:
message_data.groupby("Spam/Not_Spam").describe()

message         \
                count unique   
Spam/Not_Spam                  
ham              4825   4516   
spam              747    653   

                                                                       
                                                             top freq  
Spam/Not_Spam                                                          
ham                                       Sorry, I'll call later   30  
spam           Please call our customer service representativ...    4

In [6]:
message_text= message_data["message"].copy()

In [7]:
message_text.head(15)


0     Go until jurong point, crazy.. Available only ...
1                         Ok lar... Joking wif u oni...
2     Free entry in 2 a wkly comp to win FA Cup fina...
3     U dun say so early hor... U c already then say...
4     Nah I don't think he goes to usf, he lives aro...
5     FreeMsg Hey there darling it's been 3 week's n...
6     Even my brother is not like to speak with me. ...
7     As per your request 'Melle Melle (Oru Minnamin...
8     WINNER!! As a valued network customer you have...
9     Had your mobile 11 months or more? U R entitle...
10    I'm gonna be home soon and i don't want to tal...
11    SIX chances to win CASH! From 100 to 20,000 po...
12    URGENT! You have won a 1 week FREE membership ...
13    I've been searching for the right words to tha...
14                  I HAVE A DATE ON SUNDAY WITH WILL!!
Name: message, dtype: object

In [8]:
message_text.shape

(5572,)

In [9]:
import string
import nltk
from nltk.corpus import stopwords
def text_preprocess(text):
    text = text.translate(text.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [10]:
message_text= message_text.apply(text_preprocess)
message_text.head(20)

0     Go jurong point crazy Available bugis n great ...
1                               Ok lar Joking wif u oni
2     Free entry 2 wkly comp win FA Cup final tkts 2...
3                   U dun say early hor U c already say
4           Nah dont think goes usf lives around though
5     FreeMsg Hey darling 3 weeks word back Id like ...
6        Even brother like speak treat like aids patent
7     per request Melle Melle Oru Minnaminunginte Nu...
8     WINNER valued network customer selected receiv...
9     mobile 11 months U R entitled Update latest co...
10    Im gonna home soon dont want talk stuff anymor...
11    SIX chances win CASH 100 20000 pounds txt CSH1...
12    URGENT 1 week FREE membership å£100000 Prize J...
13    Ive searching right words thank breather promi...
14                                          DATE SUNDAY
15    XXXMobileMovieClub use credit click WAP link n...
16                                      Oh kim watching
17    Eh u remember 2 spell name Yes v naughty m

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer= TfidfVectorizer('english')
message_text_vec= vectorizer.fit_transform(message_text)

In [12]:
message_text_vec

<5572x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 47254 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.model_selection import train_test_split
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_text_vec, 
                                                        message_data['Spam/Not_Spam'], test_size=0.3, random_state=20)

In [14]:
from sklearn.naive_bayes import MultinomialNB

Spam_Clf= MultinomialNB(alpha=0.2)
Spam_Clf.fit(message_train,spam_nospam_train)

MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)

In [15]:
pred_labels= Spam_Clf.predict(message_test)

In [16]:
from sklearn.metrics import accuracy_score
score= accuracy_score(spam_nospam_test, pred_labels)

In [17]:
print(score)

0.9796650717703349
